# 以相似度为训练目标

## 生成训练数据，暂存于本地

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3b

In [0]:
import os
import cv2
import numpy as np
import random
path1 = '05wan3'
path2 = '05wan3b/0'

clocks = dict()
x_train0 = []
y_train0 = []
for filename in os.listdir(path2):
    if 'git' in filename: continue
    #print(path1)
    #ftimestr,number = path1.split('_')
    clocks[int(filename.split('_')[0])]=int(filename.split('_')[1][:-4])
    #clocks[int(ftimestr)]=int(number)
keys = clocks.keys()
filelist = os.listdir(path1)
random.shuffle(filelist)
for filename in filelist:
    if 'git' in filename or 'c.jpg' in filename: continue
    ftime = int(filename.split('_')[0])
    value = max(filter(lambda t:t<ftime,keys))
    x_train0.append(cv2.imread(os.path.join(path1,filename), cv2.IMREAD_GRAYSCALE)/255.0)
    y_train0.append(clocks[value])
    
print(len(x_train0),len(y_train0))

l = int(len(x_train0)*0.8)
x_train = x_train0[:l]
y_train = y_train0[:l]
x_test = x_train0[l:]
y_test = y_train0[l:]

# 转为onehot，分离test和train

# 装载Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

# 输入数据生成器
# 变为5张图片一叠

In [0]:
idxes = []
for i in range(10):idxes.append([])
for idx, val in enumerate(y_train):
  idxes[val].append(idx)

idxes_test = []
for i in range(10):idxes_test.append([])
for idx, val in enumerate(y_test):
  idxes_test[val].append(idx) 

# 搭建训练数据

## 搭建Overfit数据：只有十叠按序的照片

In [0]:
picnum = 5
import random
def train_generator():
  while True:
    dataout = []
    for i in range(5):dataout.append([])
    ansout = np.zeros((100,))
    for train2 in range(100):
      ans = random.randint(0,9)
      idx2 = random.sample(idxes[ans],5)
      
      #idx2 = idxes[ans][:5]
      for i in range(5):
        dataout[i].append(x_train[idx2[i]][:,:,np.newaxis])      
      ansout[train2] = ans
      #dataout.append(data)      
    yield [np.stack(data2) for data2 in dataout],ansout 


# 验证generator

In [0]:
tx,ty = next(train_generator())
print(tx[0].shape)
print(ty.shape)
import matplotlib.pyplot as plt
for i in range(5):
  plt.imshow(tx[0][i][:,:,0])
  plt.show()
  print(ty[i])


# 搭建模型和训练
现在的网络去掉dropout则可以拟合10张图片，
但是加上dropout就拟合不了。
从7张拟合5张的，则可以在两个epoch（每个epoch抽取60000次）内到80%。

继续训练20张，一个epoch就到60%。

In [0]:
picnum = 5
inputs = [tf.keras.layers.Input(shape=(12,12,1))  for i in range(picnum)]
#inputs = tf.keras.layers.Input(shape=(5,12,12,1))

conv_a1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva1 = [conv_a1(inputs[i]) for i in range(picnum)]


convs1 = tf.stack(conva1)
convf1 = [
    tf.math.reduce_max(convs1, axis=[0]),
    tf.math.reduce_mean(convs1, axis=[0]),
    tf.math.reduce_min(convs1, axis=[0]),
]

convh3 = tf.concat(convf1,axis=3)

convx3 = tf.keras.layers.Flatten()(convh3)
convy1 = tf.keras.layers.Dense(256,activation='relu')(convx3)

#keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

convy2 = tf.keras.layers.Dropout(0.5)(convy1)
convy3 = tf.keras.layers.Dense(10,activation='softmax')(convy2)

model = tf.keras.Model(inputs=inputs, outputs=convy3)

In [0]:


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# 搭建测试用generator

In [0]:

def test_generator():
  while True:
    dataout = []
    for i in range(5):dataout.append([])
    ansout = np.zeros((100,))
    for train2 in range(100):
      ans = random.randint(0,9)
      idx2 = random.sample(idxes_test[ans],5)
      
      #idx2 = idxes[ans][:5]
      for i in range(5):
        dataout[i].append(x_test[idx2[i]][:,:,np.newaxis])      
      ansout[train2] = ans
      #dataout.append(data)      
    #print('ans:',ans)  
    yield [np.stack(data2) for data2 in dataout],ansout 

# 开始训练

In [0]:
model.fit_generator(train_generator(),epochs=100,steps_per_epoch=600,validation_data=test_generator(),validation_steps=10)

# 测试正确率

In [0]:

def pred_generator():
    dataout = []
    for i in range(5):dataout.append([])
   
    
    ans = random.randint(0,9)
    idx2 = random.sample(idxes_test[ans],5)
    
    #idx2 = idxes[ans][:5]
    for i in range(5):
      plt.imshow(x_test[idx2[i]],cmap='gray')
      plt.show()
      dataout[i].append(x_test[idx2[i]][:,:,np.newaxis])      
    
    #dataout.append(data)      
    print('ans:',ans)  
    
    yield [np.stack(data2) for data2 in dataout],ans

In [0]:

#model.evaluate_generator(test_generator())

In [0]:
model.predict(pred_generator())